# Debug training
May 19, 2021

In [5]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

import socket

In [6]:
from ipywidgets import *

In [7]:
%matplotlib widget

In [11]:
base_dict={'cori':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/',
         'summit':'/autofs/nccs-svm1_home1/venkitesh/projects/cosmogan/cosmogan_pytorch/'}
facility='cori' if socket.gethostname()[:4]=='cori' else 'summit'

base_dir=base_dict[facility]

In [12]:
sys.path.append(base_dir+'/code/modules_image_analysis/')
from modules_img_analysis import *
# sys.path.append(base_dir+'/code/5_3d_cgan/1_main_code/')
# import post_analysis_pandas as post

In [15]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [28]:
dict1={'cori':{
'2d':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128_square/',
'3d':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3D/',
'3d_cgan':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'},
'summit':{'2d':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/2d/',
          '3d':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/'}}

In [48]:
# parent_dir=u.result
parent_dir=dict1[facility]['3d']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20210506*')]
dir_lst
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210506_180924_cgan_bs8_lr0.0001_nodes8', '20210506…

In [55]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/20210506_92216_bs8_lr0.0003_nodes8_spec0.1


## Plot Losses

In [56]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)
# df_metrics.tail(10)
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
f_plot_metrics(df_metrics,['hist_chi'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
f_plot_metrics(df_metrics,['hist_chi'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [39]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-0.6433697462081909


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
1957,1957.0,34.0,0.582845,0.591623,1.174469,1.124525,12.263165,11.138640,-2.028793,10.966629,-2.365599,NaN,NaN,0.276990,-0.234158,-0.721033,0.441149,0.000063,0.001
1646,1646.0,29.0,0.609501,0.783546,1.393047,1.207010,12.007258,10.800249,-2.239850,10.890207,-2.337214,NaN,NaN,0.177379,0.227939,-0.847258,0.441106,0.000063,0.001
1958,1958.0,34.0,0.631028,0.548303,1.179331,1.014080,11.881619,10.867539,-2.171142,10.919251,-2.316198,NaN,NaN,0.143202,-0.359142,-0.539608,0.453881,0.000063,0.001
1642,1642.0,29.0,0.620963,0.898771,1.519734,0.991110,12.240129,11.249020,-1.449931,11.059259,-2.235673,NaN,NaN,0.148012,0.360469,-0.492904,0.446036,0.000063,0.001
1631,1631.0,29.0,0.799103,0.769290,1.568393,0.854081,11.927874,11.073793,-2.308673,11.032211,-2.174651,NaN,NaN,-0.226182,0.166255,-0.272185,0.439907,0.000063,0.001
1629,1629.0,29.0,0.780051,0.689749,1.469800,0.815378,11.869556,11.054178,-1.666847,11.009417,-2.163078,NaN,NaN,-0.182881,-0.049058,-0.208323,0.459869,0.000063,0.001
1632,1632.0,29.0,1.229822,0.488797,1.718619,1.306310,12.586250,11.279940,-1.708414,11.009107,-2.147297,NaN,NaN,-0.934408,-0.819219,-0.962700,0.441525,0.000063,0.001
1641,1641.0,29.0,0.984981,0.883943,1.868924,0.569914,11.555775,10.985861,-2.343367,11.113537,-2.143418,NaN,NaN,-0.578710,0.401400,0.288029,0.444127,0.000063,0.001
1650,1650.0,29.0,0.735438,0.530645,1.266083,1.031423,12.100069,11.068645,-1.789464,11.007469,-2.141961,NaN,NaN,-0.022347,-0.416478,-0.580665,0.456040,0.000063,0.001
1981,1981.0,35.0,0.491810,0.721422,1.213232,1.373123,12.295521,10.922398,-1.715687,10.929399,-2.136034,NaN,NaN,0.621560,-0.033147,-1.068286,0.435850,0.000063,0.001


In [42]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
1957,1957.0,34.0,0.582845,0.591623,1.174469,1.124525,12.263165,11.138640,-2.028793,10.966629,-2.365599,NaN,NaN,0.276990,-0.234158,-0.721033,0.441149,0.000063,0.001
1646,1646.0,29.0,0.609501,0.783546,1.393047,1.207010,12.007258,10.800249,-2.239850,10.890207,-2.337214,NaN,NaN,0.177379,0.227939,-0.847258,0.441106,0.000063,0.001
1958,1958.0,34.0,0.631028,0.548303,1.179331,1.014080,11.881619,10.867539,-2.171142,10.919251,-2.316198,NaN,NaN,0.143202,-0.359142,-0.539608,0.453881,0.000063,0.001
1642,1642.0,29.0,0.620963,0.898771,1.519734,0.991110,12.240129,11.249020,-1.449931,11.059259,-2.235673,NaN,NaN,0.148012,0.360469,-0.492904,0.446036,0.000063,0.001
1631,1631.0,29.0,0.799103,0.769290,1.568393,0.854081,11.927874,11.073793,-2.308673,11.032211,-2.174651,NaN,NaN,-0.226182,0.166255,-0.272185,0.439907,0.000063,0.001
1629,1629.0,29.0,0.780051,0.689749,1.469800,0.815378,11.869556,11.054178,-1.666847,11.009417,-2.163078,NaN,NaN,-0.182881,-0.049058,-0.208323,0.459869,0.000063,0.001
1632,1632.0,29.0,1.229822,0.488797,1.718619,1.306310,12.586250,11.279940,-1.708414,11.009107,-2.147297,NaN,NaN,-0.934408,-0.819219,-0.962700,0.441525,0.000063,0.001
1641,1641.0,29.0,0.984981,0.883943,1.868924,0.569914,11.555775,10.985861,-2.343367,11.113537,-2.143418,NaN,NaN,-0.578710,0.401400,0.288029,0.444127,0.000063,0.001


## Grid plot of images

In [111]:
epoch=100
flist=glob.glob(result_dir+'/images/gen_img_epoch-{0}_step*'.format(epoch))
steps_list=[fname.split('/')[-1].split('step-')[-1].split('.')[0] for fname in flist]

print(*steps_list)

15660 15720 15610 15630 15650 15740 15710 15670 15730 15640 15680 15690 15700 15750 15620


In [106]:
# fname=flist[0]
# fname,fname.split('/')[-1].split('step-')[-1].split('.')[0]

epoch,step=5,800
fname=glob.glob(result_dir+'/images/gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
fname

'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/20210506_92216_bs8_lr0.0003_nodes8_spec0.1/images/gen_img_epoch-5_step-800.npy'

In [107]:
images=np.load(fname)[:,0,:,:]
print(images.shape)
f_plot_grid(images[:8,:,:,0],cols=4,fig_size=(8,4))

(32, 64, 64, 64)
2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …